## Citation Overrepresentation Tool
This tool extracts last authors & journals of papers you have cited and creates frequency tables so you can see who you cite the most often. It then attempts to map authors to instutions via the ORCID database. You can use it to find which journals, labs, and universities receive most of your attention.

1. Extract your citations as a .bib file. Either extract all refs from a single folder in your citation manager or extract them straight from a Word doc if you use Mendeley or Zotero using [this tool](https://rintze.zelle.me/ref-extractor/).
2. Upload your .bib file to the binder.
3. Run each cell (Shift+Enter or Play button).
4. Optional: get an ORCID API key (instructions below) to extract institutions.

In [ ]:
#imports
!pip install pybtex

from pybtex.database.input import bibtex
import glob
import pandas as pd
import requests
import re

In [ ]:
#read .bib file
ID = glob.glob('*bib')
parser = bibtex.Parser()
try:
    bib_data = parser.parse_file(ID[0])
except:
    raise ValueError("Your .bib file has non-UTF8 characters it in (like smart quotes). Please remove them & try again.")

In [ ]:
#extract author & journal names from each citation
authors = list()
for key in bib_data.entries:
    author = bib_data.entries[key].persons['author']
    first_name = author[-1].rich_first_names
    last_name = author[-1].rich_last_names
    first_name = str(first_name)[7:-3]
    last_name = str(last_name)[7:-3]

    try:
        journal = bib_data.entries[key].fields['journal']
    except:
        journal = 'Book'
    authors.append([first_name, last_name, journal])

In [ ]:
#build data frame & print
auth_df = pd.DataFrame(authors, columns=['First Name','Last Name', 'Journal'])
print('Overcited Authors')
print(auth_df.groupby(['First Name','Last Name']).size().sort_values(ascending=False).head(10))
print('\nOvercited Journals')
print(auth_df.groupby(['Journal']).size().sort_values(ascending=False).head(10))

### Optional: Get institutions from ORCID
Get an [ORCID API client ID & secret](https://support.orcid.org/hc/en-us/articles/360006897174)

You can learn more about how to [search for an ORCID](https://members.orcid.org/api/tutorial/search-orcid-registry) and [find info about an author given their ORCID](https://members.orcid.org/api/tutorial/read-orcid-records) in the API documentation.

In [ ]:
#input your client ID & key
ORCIDAPI_ID = 'YOUR ACCOUNT ID HERE'
ORCIDAPI_key = 'YOUR ACCOUNT KEY HERE'

In [ ]:
# build a request for a token
payload = {'client_id': ORCIDAPI_ID,
                   'client_secret': ORCIDAPI_key,
                   'scope': '/read-public',
                   'grant_type': 'client_credentials'
                   }
url = 'https://orcid.org/oauth/token'
headers = {'Accept': 'application/json'}
response = requests.post(url, data=payload, headers=headers, timeout=None)
response.raise_for_status()
token = response.json()['access_token']

# set up headers for searches
headers = {'Accept': 'application/vnd.orcid+json',
           'Authorization type': 'Bearer',
           'Access token': token}

In [ ]:
#find ORCIDs
#NB this section calls the API once for each author, so it takes a while
orcid_list = list()
for index, row in auth_df.iterrows():
    given = re.sub(' ','%20',row['First Name'])
    family = re.sub(' ','%20',row['Last Name'])

    #build search
    url = "https://pub.orcid.org/v3.0/search/?q=" \
        + "family-name:" + family + "+AND+given-names:" + given \
        + "&rows=1"
    auth_id = requests.get(url, headers=headers, timeout=None)

    #get first returned ORCID
    if auth_id.json()['result'] is not None:
        orcid_list.append(auth_id.json()['result'][0]['orcid-identifier']['path'])
    else:
        orcid_list.append('')

auth_df['ORCID'] = orcid_list

In [ ]:
# get institution from ORCID
#NB this section calls the API once for each author, so it takes a while
inst_list = list()
for index, row in auth_df.iterrows():
    if len(row['ORCID'])>0:
        url = "https://pub.orcid.org/v2.1/" + row['ORCID'] + "/record"
        orcid_request = requests.get(url, headers=headers, timeout=None)
        affil = orcid_request.json()['activities-summary']['employments']['employment-summary']
        if len(affil)>0:
#         print(json.dumps(var, indent=2, separators=(',', ':')))
            inst_list.append(affil[0]['organization']['name'])
        else:
            inst_list.append('Undetermined')
    else:
        inst_list.append('Undetermined')

In [ ]:
#add to df & print
auth_df['Institution'] = inst_list
print('\nOvercited Institutions')
print(auth_df.groupby(['Institution']).size().sort_values(ascending=False).head(10))